In [1]:
import pickle
import umap

import scanpy as sc
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

/Users/adr/miniconda3/envs/cmonge_updated/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
logger_path = "/Users/adr/Box/otperturb/alice_exp/notebook_logger.json"
base_exp_path = "/Users/adr/Box/otperturb/alice_exp/chemCPA/chemCPA_on_cmonge_sciplex/"
base_fig_save = "/Users/adr/Box/otperturb/alice_exp/figures/UMAPs/"
drugs_file = "/Users/adr/Box/otperturb/datasets/scrna-sciplex3/drugs.txt"
dosages=["-10", "-100", "-1000", "-10000"]

In [3]:
with open(drugs_file, 'r') as f:
    drugs = f.readlines()
drugs=[d[:-1] for d in drugs]

In [4]:
adata = sc.read_h5ad("/Users/adr/Box/otperturb/datasets/scrna-sciplex3/hvg.h5ad")
adata = adata[adata.obs["drug"].isin(drugs+["control"])]

In [5]:
umapper=umap.UMAP(random_state=1234)

In [6]:
adata.obs[["UMAP1", "UMAP2"]] = umapper.fit_transform(adata.X)

/Users/adr/miniconda3/envs/cmonge_updated/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
/var/folders/54/s3nhktxs0tz7w0f6tsj6pltc0000kp/T/ipykernel_93973/779591978.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata.obs[["UMAP1", "UMAP2"]] = umapper.fit_transform(adata.X)


## chemCPA results

In [7]:
batch_size=512

In [8]:
adata_with_split = sc.read_h5ad("/Users/adr/Box/otperturb/datasets/scrna-chemCPA-sciplex/cmonge_adata_with_chemCPA_split.h5ad")

In [13]:
for drug in drugs:
    with open(f"{base_exp_path}{drug}_ood_train_8/prediction_dict_ood.pkl", "rb") as f:
        pred_dict = pickle.load(f)

    print(drug)
    all_drug_source = adata_with_split[(adata_with_split.obs[f"{drug}_ood_train_8_split"]=="test")&(adata_with_split.obs["drug"]=="control")]
    all_drug_pred = []
    all_true = []
    all_source = []
    for cond, true_pred_degs in pred_dict.items():
        all_drug_pred.append(pd.DataFrame(true_pred_degs[1]))
        all_true.append(pd.DataFrame(true_pred_degs[0]))
        all_source.append(pd.DataFrame(all_drug_source.X.todense()))
    all_drug_pred = pd.concat(all_drug_pred, keys=pred_dict.keys(), names=("cl_drug_dose", ""))
    all_true = pd.concat(all_true, keys=pred_dict.keys(), names=("cl_drug_dose", ""))
    all_source = pd.concat(all_source, keys=pred_dict.keys(), names=("cl_drug_dose", ""))
    
    
    all_drug_pred[["UMAP1", "UMAP2"]] = umapper.transform(all_drug_pred)
    all_true[["UMAP1", "UMAP2"]] = umapper.transform(all_true)
    all_source[["UMAP1", "UMAP2"]] = umapper.transform(all_source)
    all_drug_pred["dtype"] = "trans"
    all_true["dtype"] = "target"
    all_source["dtype"] = "source"
    
    all_cells = pd.concat([all_drug_pred, all_true, all_source])
    
    all_cells = all_cells.reset_index("cl_drug_dose")
    all_cells[["cell_line", "drug", "dose"]] = [c.split("_") for c in all_cells["cl_drug_dose"]]
    all_cells["dose"] = (all_cells["dose"].astype(float) * 10000).astype(int).astype(str)
    all_cells["condition"] = all_cells["drug"] + "-" + all_cells["dose"]
    all_cells = all_cells.groupby(["dtype", "condition"]).sample(n=batch_size)

    fig, ax = plt.subplots(figsize=(5,5))
    sns.scatterplot(data=adata.obs, x="UMAP1", y="UMAP2", color="lightgrey", s=5, ax=ax)
    sns.scatterplot(data=all_cells, x="UMAP1", y="UMAP2", hue="dtype", s=20, hue_order=["source", "target", "trans"], ax=ax,
                   alpha=0.5)
    plt.title(f"{drug} OOD")
    handles, labels = ax.get_legend_handles_labels()
    plt.legend(handles, ["Source", "Target", "Transport"], title="", loc="lower left", markerscale=3)
    plt.savefig(f"{base_fig_save}chemCPA_{drug}_STT_UMAP.png", bbox_inches="tight", dpi=300)
    plt.close()
    
    for condition in all_cells["condition"].unique():
        fig, ax = plt.subplots(figsize=(5,5))
        sns.scatterplot(data=adata.obs, x="UMAP1", y="UMAP2", color="lightgrey", s=5, ax=ax)
        sns.scatterplot(data=all_cells[all_cells["condition"]==condition], x="UMAP1", y="UMAP2", hue="dtype", s=20, alpha=0.5,
                        hue_order=["source", "target", "trans"], ax=ax)
        plt.title(f"{condition} OOD")
        handles, labels = ax.get_legend_handles_labels()
        plt.legend(handles, ["Source", "Target", "Transport"], title="", loc="lower left", markerscale=3)
        plt.savefig(f"{base_fig_save}chemCPA_{condition}_STT_UMAP.png", bbox_inches="tight", dpi=300)
        plt.close()


abexinostat
belinostat
dacinostat
entinostat
givinostat
mocetinostat
pracinostat
tacedinaline
trametinib
